In [57]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained('gpt2')
model = AutoModelForCausalLM.from_pretrained('gpt2')

In [ ]:
# prompt = 'the dog walked the man to the park'

# tokens = tokenizer.encode(prompt, return_tensors='pt')
# print(tokens)
# print(tokenizer.tokenize(prompt))
# outputs = model(tokens, output_attentions=True)
# print(len(outputs.attentions))
# outputs.attentions[0].shape

In [ ]:
# from bertviz import model_view, head_view

# model_view(outputs.attentions, tokenizer.convert_ids_to_tokens(tokens[0]))
# head_view(outputs.attentions, tokenizer.convert_ids_to_tokens(tokens[0]))

In [ ]:
# Goal:
# 1. Take a prompt.
# 2. Generate an answer to the question.
# 3. Quantify where attention was paid on each token of the answer.
# 4. Visualize that attention. Visualization should work for long passages.

In [107]:
def inspect_shape(name, obj):
    # Since this is the top-level inspect call, print the name of the variable.
    print(f'{name}: ', end='')
    _inspect_shape(obj, len(name) + 2)  # + 2 for ': '


def _class_name(obj):
    return type(obj).__name__


def _inspect_shape(obj, indent):
    """ 
    Print the given obj. Does not print preceding name or preceding indentation.
    Param `indent` should be the indentation level where this obj is being printed. Will be used to print nested properties if necessary.
    """

    # Base Cases (sequences with known-typed contents, tensors, other objects or primitives)
    if isinstance(obj, str):
        print(f'{_class_name(obj)}[len={len(obj)}]')
    elif isinstance(obj, torch.Tensor):
        print(f'{_class_name(obj)}[shape={list(obj.shape)}, sum={obj.sum()}]')
    else:

        # Dict where contents should be recursively inspected
        try:
            print(f'{_class_name(obj)}[len={len(obj.items())}]')
            for key, val in obj.items():
                print(' ' * (indent + 2) + f'{key}: ', end='')
                _inspect_shape(val, indent + len(key) + 2)  # + 2 for ': '
        except (TypeError, AttributeError):
            # Sequence where contents should be recursively inspected
            try:
                print(f'{_class_name(obj)}[len={len(obj)}]')
                for i in range(min(len(obj), 3)):
                    print(' ' * (indent + 2) + f'{i}: ', end='')
                    # + 2 for ': '
                    _inspect_shape(obj[i], indent + len(str(i)) + 2)
            except (TypeError, AttributeError):
                # Default case if nothing else works
                print(f'{_class_name(obj)}')

In [193]:
import altair as alt
import pandas as pd
alt.data_transformers.enable("vegafusion")


def attn_viz(prompt_tokens, generated_sequence_tokens, generated_token_avg_attns, width, scale):
    """
    prompt_tokens should be the Tensor of token IDs from the prompt, 1 x (prompt len)
    generated_sequence should be the raw sequence from huggingface .generate(), including both prompt and generated tokens.
    generated_token_avg_attns should be a sequence with len(generated tokens), where each element is a sequence with len(tokens through generated token, including prompt)
    """
    df = _prepare_attn_df(prompt_tokens, generated_sequence_tokens,
                          generated_token_avg_attns)
    _show_attn_chart(df, width, scale)


def _printable_token_text(token):
    return tokenizer.decode(token).replace('\n', '\\n')


def _prepare_attn_df(prompt_tokens, generated_sequence_tokens, generated_token_avg_attns):
    inspect_shape('generated_sequence_tokens', generated_sequence_tokens)
    inspect_shape('list(generated_sequence_tokens)',
                  list(generated_sequence_tokens))
    rows = []

    num_prompt_tokens = prompt_tokens.shape[1]

    # TODO: Make this much more efficient.
    # - Tensor iteration
    # - Don't both generating rows for attn_from a prompt token
    for token_idx, token_id_tensor in enumerate(generated_sequence_tokens):
        token_id = token_id_tensor.item()
        for attn_from_idx in range(len(generated_sequence_tokens)):
            if attn_from_idx < num_prompt_tokens:
                attn = 0
            else:
                generated_idx = attn_from_idx - num_prompt_tokens
                generated_token_avg_attn = generated_token_avg_attns[generated_idx]
                if token_idx < generated_token_avg_attn.shape[0]:
                    attn = generated_token_avg_attn[token_idx].item()
                else:  # If this token is in the future it cannot receive attention from a past token
                    attn = 0

            rows.append({
                'token_idx': token_idx,
                'token_id': token_id,
                'token_text': _printable_token_text(token_id),
                'attn_from_idx': attn_from_idx,
                'attn': attn
            })

    df = pd.DataFrame(rows)
    display(df)

    # for source_token_idx, generated_token_avg_attn in enumerate(generated_token_avg_attns):
    #     for target_token_idx, target_token_attn in enumerate(list(generated_token_avg_attn)):
    #         rows.append({
    #             'source_token_idx': source_token_idx,
    #             'source_token_id': generated_sequence_tokens
    #         })
    # df = pd.DataFrame({'token_id': generated_sequence_tokens, 'token_text': map(
    #     _printable_token_text, generated_sequence_tokens)})

    # attn_from_cols = [f'token_attn_from_{i}' for i in range(
    #     len(generated_sequence_tokens))]
    # df = df.reindex(columns=['token_id', 'token_text'] +
    #                 attn_from_cols, fill_value=0.0)
    # df['token_idx'] = df.index
    # df['token_attn_col'] = df['token_idx'].apply(
    #     lambda idx: 'token_attn_col_{}'.format(idx))

    # # Generated tokens give attention to preceding tokens
    # # TODO: Make this more efficient, maybe with an entire matrix instead of setting each vector?
    # for token_idx, generated_token_avg_attn in enumerate(generated_token_avg_attns):
    #     # inspect_shape('generated_token_avg_attn', generated_token_avg_attn)
    #     # print('prompt tokens shape', prompt_tokens.shape)
    #     absolute_token_idx = prompt_tokens.shape[1] + token_idx
    #     df.loc[:len(generated_token_avg_attn) - 1,
    #            f'token_attn_from_{absolute_token_idx}'] = list(generated_token_avg_attn)
    # # display(df)
    return df


def _show_attn_chart(df, width, scale):
    """
    df should have columns:
      token_idx: absolute index of token within sequence, including prompt tokens
      token_id: int representation of token
      token_text: string representation of token
      attn_from_idx: 
      attn: 
    width in pixels, of total visualization
    scale in float, 1.0 is normal scale (48px x 24px boxes)
    """

    chart_df = df.copy()

    # Add x and y information to the tokens based on the calculated number of tokens that can fit the given width
    tokens_w = width // (scale * 48)
    chart_df['x'] = chart_df['token_idx'].apply(lambda idx: idx % tokens_w)
    chart_df['y'] = chart_df['token_idx'].apply(lambda idx: idx // tokens_w)

    # display(chart_df)

    selected_token = alt.selection_point(
        on='mouseover', name='SelectedToken', fields=['token_idx'], empty=False)

    selected_token_from = selected_token.project()

    base = alt.Chart(chart_df).encode(
        x=alt.X('x:N').title('').axis(labels=False, ticks=False),
        y=alt.Y('y:N').title('').axis(labels=False, ticks=False)
    ).properties(
        width=width,
        # Number of columns times height of each box
        height=(len(df.index) // tokens_w) * (scale * 24)
        # ).transform_calculate(
        #     selected_attn=f'datum[{selected_token.name}]'
        # ).add_params(
        #     selected_token
    )

    highlight = base.mark_rect().encode(
        color=alt.Color('selected_attn:Q').scale(
            scheme='tealblues').legend(None)
        # color=alt.condition(
        #     alt.datum.y > selected_token.y,
        #     alt.value('red'),
        #     alt.value('lightgray')
        # )
        # ).transform_calculate(
        #     selected_attn=f'datum[{selected_token.name}]'
        # ).add_params(
        #     highlight_param,
        #     selected_token
    ).transform_filter(
        alt.selected_token
    )

    text = base.mark_text(baseline='middle', fontSize=(scale * 10)).encode(
        text=alt.condition(selected_token, alt.value('###'), 'token_text:N')
    ).add_params(
        selected_token
    )

    display(highlight + text)


def average_token_attn(token_attn):
    # TODO: Make this more efficient using torch tensor operations instead of splitting lists
    layer_avg_attns = []
    for layer_idx, layer_attn in enumerate(token_attn):
        # print('layer_idx', layer_idx)
        # Take mean across model's attention heads
        layer_avg_attn = layer_attn.squeeze(0).mean(dim=0)
        # inspect_shape('layer_avg_attn', layer_avg_attn)
        layer_avg_attn_cleaned = torch.concat([
            torch.tensor([0.]),  # First entry is null attention, set it to 0
            # Remove all tokens except the most recent (since the first token generated has entire prompts' worth of attention generated) and remove the first entry
            layer_avg_attn[-1][1:],
            torch.tensor([0.]),  # Add a 0 for the current token itself
        ])
        # inspect_shape('layer_avg_attn_cleaned', layer_avg_attn_cleaned)
        layer_avg_attn_normalized = layer_avg_attn_cleaned / layer_avg_attn_cleaned.sum()
        # inspect_shape('layer_avg_attn_normalized', layer_avg_attn_normalized)
        layer_avg_attns.append(layer_avg_attn_normalized)
    # inspect_shape('layer_avg_attns', layer_attn)
    return torch.stack(layer_avg_attns).mean(dim=0)


def generate_and_attn(prompt):
    prompt_tokens = tokenizer.encode(prompt, return_tensors='pt')
    # inspect_shape('tokens', prompt_tokens)

    outputs = model.generate(prompt_tokens, max_new_tokens=48,
                             output_attentions=True, return_dict_in_generate=True, pad_token_id=tokenizer.eos_token_id)
    # inspect_shape('outputs', outputs)
    # token_avg_attn = average_token_attn(outputs.attentions[1])
    # inspect_shape('token_avg_attn', token_avg_attn)
    # print(token_avg_attn)
    # for token in outputs.sequences[0]:
    #     print(token, tokenizer.decode(token), end=', ')
    generated_token_avg_attns = list(map(
        average_token_attn, outputs.attentions))
    inspect_shape('generated_token_avg_attns', generated_token_avg_attns)

    # inspect_shape('outputs.sequences[0]', outputs.sequences[0])
    attn_viz(prompt_tokens,
             outputs.sequences[0], generated_token_avg_attns, 800, 2.0)


PROMPT = """
The 2008 Summer Olympics torch relay was run from March 24 until August 8, 2008, prior to the 2008 Summer Olympics, with the theme of "one world, one dream". The torch relay took place over 45km with 18 total runners.

Q: What was the theme?
A:
""".strip()

generate_and_attn(PROMPT)

generated_token_avg_attns: list[len=48]
                             0: Tensor[shape=[61], sum=1.0]
                             1: Tensor[shape=[62], sum=1.0]
                             2: Tensor[shape=[63], sum=1.0]
generated_sequence_tokens: Tensor[shape=[108], sum=454166]
list(generated_sequence_tokens): list[len=108]
                                   0: Tensor[shape=[], sum=464]
                                   1: Tensor[shape=[], sum=3648]
                                   2: Tensor[shape=[], sum=10216]


,token_idx,token_id,token_text,attn_from_idx,attn
0,0,464,The,0,0.0
1,0,464,The,1,0.0
2,0,464,The,2,0.0
3,0,464,The,3,0.0
4,0,464,The,4,0.0
...,...,...,...,...,...
11659,107,625,over,103,0.0
11660,107,625,over,104,0.0
11661,107,625,over,105,0.0
11662,107,625,over,106,0.0


ValueError: Expression compilation error: No signal named SelectedToken in evaluation scope
    Context[0]: Failed to get node value


alt.LayerChart(...)